Create Azure SQL Virtual Machine
============================================

Description
-----------
For more information about other Azure PowerShell options for creating SQL VMs, see the <a href="https://docs.microsoft.com/en-us/azure/virtual-machines/windows/sql/virtual-machines-windows-ps-sql-create">Provisioning guide for SQL Server VMs with Azure PowerShell</a>. See also <a href="https://docs.microsoft.com/en-us/azure/virtual-machines/windows/sql/quickstart-sql-vm-create-powershell">Quickstart guide</a> to creating a SQL Server VM with Azure PowerShell.


Steps of this procedure include:
1. Connect to Azure subscription
1. Provision resource group for SQL VM migration
2. Create a storage account
3. Configure Network Settings
3. Provision SQL VM
4. Configure SQL VM IaaS agent

#### <b> *Enter all the values in quotes *</b>

In [1]:
# Resource Group  
$ResourceGroupName  =   "aaskhan"                             # Name of the resource group to create in the current subscription
$Location           =   "West US 2"                             # see Appendix for a list of location settings

# Compute
$VMName             =   "aasim_sqlvm"                             # VM to create
$PublisherName      =   "MicrosoftSQLServer"                             # Name of Publisher, Default would be 'MicrosoftSQLServer'
$Version            =   "latest"                             # Version of VM, Default would be 'latest'

# Storage
$StorageAccountName =   $ResourceGroupName + "_storage"
$StorageSku         =   "Standard_LRS"                             # Choose your storage sku (see appendix)
$StorageName        =   "sqlstorage12121" + (Get-Random -Minimum 1 -Maximum 100)

# VM Password
$secureVMPassword   =   "aasimkhan30"                             # Create the password  for VM.

## Connect to Azure Account

In [2]:
Connect-AzAccount


Account               SubscriptionName   TenantId                             Environment
-------               ----------------   --------                             -----------
aaskhan@microsoft.com AnE.ExP.Production 72f988bf-86f1-41af-91ab-2d7cd011db47 AzureCloud 




## Get Subscription
Below command will open a _Dialouge Box_ with list of subscriptions. Selecting one of those will set that subscription for rest of the commands.

In [3]:
$subscription = Get-AzSubscription | Out-GridView -PassThru
Set-AzContext -SubscriptionName $subscription


Name                                     Account             SubscriptionName    Environment         TenantId          
----                                     -------             ----------------    -----------         --------          
ClientToolsInfra_670062 (88d5392f-a34... aaskhan@microsof... ClientToolsInfra... AzureCloud          72f988bf-86f1-4...




## Create Azure Resource Group

In [4]:
# Create Azure resource group, if necessary
$rg = Get-AzResourceGroup | Where ResourceGroupName -eq $ResourceGroupName

if (!$rg)
{
    # Need to create a new resource group
    Write-Output "Resource Group $ResourceGroupName does not exist. Creating..."
    $rg = New-AzResourceGroup -Name $ResourceGroupName -Location $Location
}

## Create a storage account
VMs require storage resources for OS, SQL data and logs. Create a new storage account as a place for it.

In [7]:
$StorageAccount = Get-AzStorageAccount | Where StorageAccountName -eq $StorageAccountName

if (!$StorageAccount)
{
    Write-Output "Storage Account $StorageName does not exist. Creating..."
    $StorageAccount = New-AzStorageAccount -ResourceGroupName $ResourceGroupName -Name $StorageName -SkuName $StorageSku -Kind "Storage" -Location $Location
}

Storage Account sqlstorage1212186 does not exist. Creating...


## Configure network settings
Create a virtual network, subnet, and a public IP address. These resources are used to provide network connectivity to the virtual machine and connect it to the internet.

In [8]:
$SubnetName = $ResourceGroupName + "subnet"
$VnetName = $ResourceGroupName + "vnet"
$PipName = $ResourceGroupName + $(Get-Random)

# Create a subnet configuration
$SubnetConfig = New-AzVirtualNetworkSubnetConfig -Name $SubnetName -AddressPrefix 192.168.1.0/24

# Create a virtual network
$Vnet = New-AzVirtualNetwork -ResourceGroupName $ResourceGroupName -Location $Location `
   -Name $VnetName -AddressPrefix 192.168.0.0/16 -Subnet $SubnetConfig

# Create a public IP address and specify a DNS name
$Pip = New-AzPublicIpAddress -ResourceGroupName $ResourceGroupName -Location $Location `
   -AllocationMethod Static -IdleTimeoutInMinutes 4 -Name $PipName


 Old Way : -ResourceId
 New Way : -NatGatewayId

 Old Way : -InputObject
 New Way : -NatGateway
information on breaking changes in Azure PowerShell.
New-AzVirtualNetwork : Windows PowerShell is in NonInteractive mode. Read and Prompt functionality is not available.
At line:8 char:9
+ $Vnet = New-AzVirtualNetwork -ResourceGroupName $ResourceGroupName -L ...
+         ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    + CategoryInfo          : CloseError: (:) [New-AzVirtualNetwork], PSInvalidOperationException
    + FullyQualifiedErrorId : Microsoft.Azure.Commands.Network.NewAzureVirtualNetworkCommand
 

 Old Way : Zone = [] means the Standard Public IP address is zone-redundant
 New Way : Zone = [] means the Standard Public IP has no zones. If you want to create a zone-redundant Public IP 
address, please specify all the zones in the region. For example, Zone = ["1", "2", "3"].To learn more visit 
aka.ms/standardpublicip
information on breaking changes in Azure PowerShel

2. Create a network security group. Configure rules to allow remote desktop (RDP) and SQL Server connections.

In [9]:
# Rule to allow remote desktop (RDP)
$NsgRuleRDP = New-AzNetworkSecurityRuleConfig -Name "RDPRule" -Protocol Tcp `
   -Direction Inbound -Priority 1000 -SourceAddressPrefix * -SourcePortRange * `
   -DestinationAddressPrefix * -DestinationPortRange 3389 -Access Allow

#Rule to allow SQL Server connections on port 1433
$NsgRuleSQL = New-AzNetworkSecurityRuleConfig -Name "MSSQLRule"  -Protocol Tcp `
   -Direction Inbound -Priority 1001 -SourceAddressPrefix * -SourcePortRange * `
   -DestinationAddressPrefix * -DestinationPortRange 1433 -Access Allow

# Create the network security group
$NsgName = $ResourceGroupName + "nsg"
$Nsg = New-AzNetworkSecurityGroup -ResourceGroupName $ResourceGroupName `
   -Location $Location -Name $NsgName `
   -SecurityRules $NsgRuleRDP,$NsgRuleSQL

3. Create the network interface.

In [10]:
$InterfaceName = $ResourceGroupName + "int"
$Interface = New-AzNetworkInterface -Name $InterfaceName `
   -ResourceGroupName $ResourceGroupName -Location $Location `
   -SubnetId $VNet.Subnets[0].Id -PublicIpAddressId $Pip.Id `
   -NetworkSecurityGroupId $Nsg.Id

Cannot index into a null array.
At line:3 char:1
+ $Interface = New-AzNetworkInterface -Name $InterfaceName `
+ ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    + CategoryInfo          : InvalidOperation: (:) [], RuntimeException
    + FullyQualifiedErrorId : NullArray
 


## Create the SQL VM
1. Define your credentials to sign in to the VM. The username is "azureadmin". Make sure you change <password> before running the command.

In [13]:
# Define a credential object
$SecurePassword = ConvertTo-SecureString $secureVMPassword `
   -AsPlainText -Force
$Cred = New-Object System.Management.Automation.PSCredential ("azureadmin", $securePassword)

### Appendix: Azure SQL VM Offers
Run the following command to get updated list of offers for Microsoft SQL Server in your location. These settings can be used for the OfferName associated with this migration. Configure it for your purposes.

Note that the SQL Version is first then appended with an operating system version. E.g.: "WS2019" means Windows Server 2019. Along with various versions of Windows Servers, there are also enterprise Linux versions such as RedHat Enterprise, Suse Enterprise, and Ubuntu. Some versions are BYOL (Bring Your Own License) aka <a href="https://azure.microsoft.com/en-us/pricing/hybrid-benefit/">Hybrid Benefit</a>.

In [14]:
Get-AzVMImageOffer -Location $Location -Publisher $PublisherName | Select Offer


Offer                        
-----                        
SQL2008R2SP3-WS2008R2SP1     
sql2008r2sp3-ws2008r2sp1-byol
SQL2012SP3-WS2012R2          
SQL2012SP3-WS2012R2-BYOL     
SQL2012SP4-WS2012R2          
SQL2012SP4-WS2012R2-BYOL     
SQL2014SP2-WS2012R2          
SQL2014SP2-WS2012R2-BYOL     
sql2014sp3-ws2012r2          
sql2014sp3-ws2012r2-byol     
SQL2016SP1-WS2012R2          
SQL2016SP1-WS2016            
SQL2016SP1-WS2016-BYOL       
SQL2016SP2-WS2012R2          
SQL2016SP2-WS2016            
SQL2016SP2-WS2016-BYOL       
sql2016sp2-ws2019            
sql2016sp2-ws2019-byol       
SQL2017-RHEL7                
SQL2017-SLES12SP2            
SQL2017-Ubuntu1604           
SQL2017-WS2016               
SQL2017-WS2016-BYOL          
sql2017-ws2019               
sql2017-ws2019-byol          
SQL2019-RHEL7                
sql2019-rhel8                
sql2019-sles12sp5            
SQL2019-Ubuntu1604           
sql2019-ubuntu1804           
SQL2019-WS2016               
sql2019-w

2. Create a virtual machine configuration object and then create the VM. The following command creates a SQL Server 2017 Developer Edition VM on Windows Server 2016.

In [ ]:
# Create a virtual machine configuration

$VMConfig = New-AzVMConfig -VMName $VMName -VMSize Standard_DS13_V2 |
   Set-AzVMOperatingSystem -Windows -ComputerName $VMName -Credential $Cred -ProvisionVMAgent -EnableAutoUpdate |
   Set-AzVMSourceImage -PublisherName "MicrosoftSQLServer" -Offer "SQL2017-WS2016" -Skus "SQLDEV" -Version "latest" |
   Add-AzVMNetworkInterface -Id $Interface.Id



## Create Virtual Machine

In [ ]:
# Create the VM
New-AzVM -ResourceGroupName $ResourceGroupName -Location $Location -VM $VMConfig

## Install the SQL IaaS Agent
To get portal integration and SQL VM features, you must install the SQL Server IaaS Agent Extension. To install the agent on the new VM, run the following command after the VM is created.

In [ ]:
Set-AzVMSqlServerExtension -ResourceGroupName $ResourceGroupName -VMName $VMName -name "SQLIaasExtension" -version "2.0" -Location $Location

## Appendices
These sections are not vital for the execution of this notebook but provide valuable background information.

### Appendix: Locations
See the <a href="https://azure.microsoft.com/en-us/global-infrastructure/locations/">Azure locations</a> page for a complete list of Azure regions along with their general physical location. The following is a list of common North American location settings for this guide:

#### US Regions
| Setting | Location |
| ------------ | --------- |
| Central US | Iowa |
| East US | Virginia |
| East US 2 | Virginia |
| North Central US | Illinois |
| South Central US | Texas |
| West US 2 | Washington |
| West Central US | Wyoming |
| West US | California | 
| Canada Central | Toronto |
| Canada East | Quebec City |
| Brazil South | Sao Paulo |
| Mexico Central | Queretaro |


### Appendix: Storage Skus
Use these as values for provisioning storage skus. 

Data for table taken from <a href="https://docs.microsoft.com/en-us/rest/api/storagerp/srp_sku_types">SKU Types</a> page but is subject to change. Not all skus are listed here. SKU type names are case-sensitive.

| Name | Description |
| -----| ----------- |
| Standard_LRS | standard locally redundant storage |
| Standard_GRS | standard geo-replicated storage |
| Standard_RAGRS | standard read-access geo replicated storage |
| Standard_ZRS | standard zone redundant storage |
| Premium_LRS | Premium - provisioned IO locally redundant |
| Premium_ZRS | Premium - provisioned IO locally redundant |
